# Onboarding Multiple CSVs

**Prerequisite**:

- Installed Neat, see [Installation](../../gettingstarted/installation.html)
- Launched a notebook environment.
- Familiar with the `NeatSession` object, see [introduction](../introduction/introduction.html)
- Access to `NeatEngine`.

In this tutorial, we will load data from two `csv` files, connect the data, infer a data model from the data and push the model with data to CDF.

## Reading Metadata

We will start by instansiating a `NeatSession` and read the data from multiple URLs.

In [1]:
from cognite.neat import NeatSession, get_cognite_client

In [2]:
# Note that we use Oxigraph in this example, this will not work in a CDF notebook
neat = NeatSession(get_cognite_client(".env"), storage="oxigraph")

Found .env file in repository root. Loaded variables from .env file.
Neat Engine 2.0.3 loaded.


In [3]:
base_url = "https://apps-cdn.cogniteapp.com/toolkit/publicdata/"

asset = "assets.Table.csv"
activity = "workitem.Table.csv"

neat.read.csv(f"{base_url}{asset}", type="Asset", primary_key="WMT_TAG_GLOBALID")
neat.read.csv(f"{base_url}{activity}", type="Activity", primary_key="sourceId")

In [4]:
neat

Studying the output above, we see that we succesfully read the assets and activities.

## Connecting Data

By studying the source data, we notice that the `WORKORDER_ITEMNAME` column in the activity CSV is referencing the `WMT_TAG_NAME` in the asset CSV.

We use neat to connect the activities to assets, through a new property we call `asset` in the activity.

In [5]:
source = ("Activity", "WORKORDER_ITEMNAME")
target = ("Asset", "WMT_TAG_NAME")
connection = "asset"

In [6]:
neat.prepare.instances.make_connection_on_exact_match(source, target, connection, limit=None)

Adds property that contains id of reference to all references of given class in Rules: 100%|█████████████████████████████████████████████████████████████████████████████████████| 12914/12914 [00:00<00:00, 16266.93it/s]


## Infer Data Model

We can infer a data model from data in the `NeatSession` by calling `.infer()`.

In [7]:
neat.infer()

[ResourceRegexViolationWarning(identifier='source.1', resource_type='Property', location='Properties sheet, Property column', regex='(?!^(property|space|externalId|createdTime|lastUpdatedTime|deletedTime|edge_id|node_id|project_id|property_group|seq|tg_table_name|extensions)$)(^[a-zA-Z][a-zA-Z0-9_]{0,253}[a-zA-Z0-9]?$)'),
 ResourceRegexViolationWarning(identifier='externalId', resource_type='Property', location='Properties sheet, Property column', regex='(?!^(property|space|externalId|createdTime|lastUpdatedTime|deletedTime|edge_id|node_id|project_id|property_group|seq|tg_table_name|extensions)$)(^[a-zA-Z][a-zA-Z0-9_]{0,253}[a-zA-Z0-9]?$)')]

In [9]:
neat.inspect.issues()

### 2 issues found

  * **ResourceRegexViolationWarning**: The Property with identifier source.1 in the Properties sheet, Property column is violating
    the CDF regex (?!^(property|space|externalId|createdTime|lastUpdatedTime|deletedTime|edge_id|node_id|project_id|property_group|seq|tg_table_name|extensions)$)(^[a-zA-Z][a-zA-Z0-9_]{0,253}[a-zA-Z0-9]?$). This will lead to errors when converting to DMS data model.
    
Fix: Either export the data model and make the necessary changes manually or run fix.data_model.cdf_compliant_external_ids.
  * **ResourceRegexViolationWarning**: The Property with identifier externalId in the Properties sheet, Property column is violating
    the CDF regex (?!^(property|space|externalId|createdTime|lastUpdatedTime|deletedTime|edge_id|node_id|project_id|property_group|seq|tg_table_name|extensions)$)(^[a-zA-Z][a-zA-Z0-9_]{0,253}[a-zA-Z0-9]?$). This will lead to errors when converting to DMS data model.
    
Fix: Either export the data model and make the necessary changes manually or run fix.data_model.cdf_compliant_external_ids.

We notice that one of the properies in the `Activity` contains a `.` which is illegal for CDF data models. Looking at the type hint we see that we can usea  prepare method to fix this issue.

In [10]:
neat.fix.data_model.cdf_compliant_external_ids()

[]

The model is now ready to be converted to the physical format

In addition, we see that one of the properties is named `externalId` which is a reserved word. In the conversion, we skip this

In [11]:
neat.convert(reserved_properties="skip")

Rules converted to dms.
You can inspect the issues with the .inspect.issues(...) method.


[NeatValueWarning(value='Property externalId is a reserved property in DMS. Skipping...')]

In [12]:
neat.inspect.issues()

### 1 issues found

  * **NeatValueWarning**: Property externalId is a reserved property in DMS. Skipping...

## Inspect Data Model

We use the `.show` to inspect the data model and see that the activity is connected to the asset

In [13]:
neat.set.data_model_id(("sp_doctrino2", "DoctrinoAssetActivityModel", "v1"))

[]

In [14]:
neat.show.data_model()

http_purl.org_cognite_neat_data-model_verified_physical_sp_doctrino2_DoctrinoAssetActivityModel_v1.html


Inspecting the model, we see that we have successfully linked the activity with the asset.

## Publishing Data Model to CDF

Now we are ready to publish this to CDF.

In [17]:
neat.to.cdf.data_model()

You can inspect the details with the .inspect.outcome.data_model(...) method.


[UploadResult(name='spaces', error_messages=[], issues=[], created=set(), upserted=set(), deleted=set(), changed=set(), unchanged={'sp_doctrino2'}, skipped=set(), failed_created=set(), failed_upserted=set(), failed_changed=set(), failed_deleted=set()),
 UploadResult(name='containers', error_messages=[], issues=[], created=set(), upserted=set(), deleted=set(), changed=set(), unchanged={ContainerId(space='sp_doctrino2', external_id='Activity'), ContainerId(space='sp_doctrino2', external_id='Asset')}, skipped=set(), failed_created=set(), failed_upserted=set(), failed_changed=set(), failed_deleted=set()),
 UploadResult(name='views', error_messages=[], issues=[], created=set(), upserted=set(), deleted=set(), changed=set(), unchanged={ViewId(space='sp_doctrino2', external_id='Activity', version='v1'), ViewId(space='sp_doctrino2', external_id='Asset', version='v1')}, skipped=set(), failed_created=set(), failed_upserted=set(), failed_changed=set(), failed_deleted=set()),
 UploadResult(name='data_models', error_messages=[], issues=[], created=set(), upserted=set(), deleted=set(), changed=set(), unchanged={DataModelId(space='sp_doctrino2', external_id='DoctrinoAssetActivityModel', version='v1')}, skipped=set(), failed_created=set(), failed_upserted=set(), failed_changed=set(), failed_deleted=set()),
 UploadResult(name='nodes', error_messages=[], issues=[], created=set(), upserted=set(), deleted=set(), changed=set(), unchanged=set(), skipped=set(), failed_created=set(), failed_upserted=set(), failed_changed=set(), failed_deleted=set())]

<img src="../../artifacts/figs/connecting_metadata_published_dm.png" width="600">

## Populating Data Model

Neat keeps track of the data, so we can immidiately populate this data model with the original data

In [16]:
neat.to.cdf.instances()

INFO | 2025-01-26 14:33:59,464 | Staring DMSLoader and will process 2 views.
INFO | 2025-01-26 14:33:59,476 | Starting ViewId(space='sp_doctrino2', external_id='Activity', version='v1') 1/2.
Loading ViewId(space='sp_doctrino2', external_id='Activity', version='v1'): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 13002/13002 [00:40<00:00, 322.44it/s]
INFO | 2025-01-26 14:34:39,807 | Finished ViewId(space='sp_doctrino2', external_id='Activity', version='v1').
INFO | 2025-01-26 14:34:39,941 | Starting ViewId(space='sp_doctrino2', external_id='Asset', version='v1') 2/2.
Loading ViewId(space='sp_doctrino2', external_id='Asset', version='v1'): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 1103/1103 [00:03<00:00, 354.32it/s]
INFO | 2025-01-26 14:34:43,059 | Finished ViewId(space='sp_doctrino2', external_id='Asset', version='v1').


You can inspect the details with the .inspect.outcome.instances(...) method.


[UploadResult(name='Activity', error_messages=[], issues=[], created=set(), upserted=set(), deleted=set(), changed={NodeId(space='sp_doctrino2_instances', external_id='4359803836140490'), NodeId(space='sp_doctrino2_instances', external_id='7644067449119447'), NodeId(space='sp_doctrino2_instances', external_id='1423967727760067'), NodeId(space='sp_doctrino2_instances', external_id='766561245809947'), NodeId(space='sp_doctrino2_instances', external_id='5588677035885395'), NodeId(space='sp_doctrino2_instances', external_id='4860657222431692'), NodeId(space='sp_doctrino2_instances', external_id='4688964855513910'), NodeId(space='sp_doctrino2_instances', external_id='2087563993408034'), NodeId(space='sp_doctrino2_instances', external_id='7689031503600099'), NodeId(space='sp_doctrino2_instances', external_id='997363156378224'), NodeId(space='sp_doctrino2_instances', external_id='4410364167108510'), NodeId(space='sp_doctrino2_instances', external_id='5871544985172602'), NodeId(space='sp_doctrino2_instances', external_id='7566149269344570'), NodeId(space='sp_doctrino2_instances', external_id='5842714601485572'), NodeId(space='sp_doctrino2_instances', external_id='6322444122956293'), NodeId(space='sp_doctrino2_instances', external_id='1501827087079372'), NodeId(space='sp_doctrino2_instances', external_id='2922965111627301'), NodeId(space='sp_doctrino2_instances', external_id='2248231574464912'), NodeId(space='sp_doctrino2_instances', external_id='2820342824790004'), NodeId(space='sp_doctrino2_instances', external_id='6579342936501653'), NodeId(space='sp_doctrino2_instances', external_id='2060067549697529'), NodeId(space='sp_doctrino2_instances', external_id='245191024497460'), NodeId(space='sp_doctrino2_instances', external_id='4697819965513113'), NodeId(space='sp_doctrino2_instances', external_id='8979891041271860'), NodeId(space='sp_doctrino2_instances', external_id='7550171278884367'), NodeId(space='sp_doctrino2_instances', external_id='978667960977710'), NodeId(space='sp_doctrino2_instances', external_id='4232323620983219'), NodeId(space='sp_doctrino2_instances', external_id='5916232031197542'), NodeId(space='sp_doctrino2_instances', external_id='2313918980707600'), NodeId(space='sp_doctrino2_instances', external_id='6868598802072732'), NodeId(space='sp_doctrino2_instances', external_id='335331364920664'), NodeId(space='sp_doctrino2_instances', external_id='4663341417943783'), NodeId(space='sp_doctrino2_instances', external_id='109061601644100'), NodeId(space='sp_doctrino2_instances', external_id='6169527674628716'), NodeId(space='sp_doctrino2_instances', external_id='7639518534344464'), NodeId(space='sp_doctrino2_instances', external_id='3237620621144535'), NodeId(space='sp_doctrino2_instances', external_id='6844261998062089'), NodeId(space='sp_doctrino2_instances', external_id='3208334119700379'), NodeId(space='sp_doctrino2_instances', external_id='7804803718457900'), NodeId(space='sp_doctrino2_instances', external_id='2756074991761761'), NodeId(space='sp_doctrino2_instances', external_id='113704318222632'), NodeId(space='sp_doctrino2_instances', external_id='1487305320446166'), NodeId(space='sp_doctrino2_instances', external_id='4303135691491830'), NodeId(space='sp_doctrino2_instances', external_id='5075902660457840'), NodeId(space='sp_doctrino2_instances', external_id='2211260170920760'), NodeId(space='sp_doctrino2_instances', external_id='5534754477881433'), NodeId(space='sp_doctrino2_instances', external_id='4417901951746188'), NodeId(space='sp_doctrino2_instances', external_id='2002379120909356'), NodeId(space='sp_doctrino2_instances', external_id='1396441070435509'), NodeId(space='sp_doctrino2_instances', external_id='492470817896500'), NodeId(space='sp_doctrino2_instances', external_id='6649658657422061'), NodeId(space='sp_doctrino2_instances', external_id='4561857008798131'), NodeId(space='sp_doctrino2_instances', external_id='152107455592809'), NodeId(space='sp_doctrino2_instances', external_id='2314559045019250'), NodeI

<img src="../../artifacts/figs/connecting_metadata_populated.png" width="1200">